In [1]:
# -*- coding: utf-8 -*-

# Get filename from the absolute file path
def getfilename(abs_file_path):
    return abs_file_path.split('/')[-1]

In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Show an Image 
def showImage(abs_file_path):
    image = cv2.imread(abs_file_path)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    plt.title(getfilename(abs_file_path))
    plt.show()

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Absolute file path of  training data images
train_data = np.array([])

# Target Label of training data
train_target = np.array([])

# Get all absolute file path and target labels in the train directory

path = os.getcwd()+"/train"

for root, dirs, files in os.walk(path):
    for filename in files:
        train_target = np.append(train_target, filename.split('_')[0])
        train_data = np.append(train_data, os.path.join(root, filename))
        
showImage(train_data[0])

# Applying SIFT detection to train data
sift_train_data = np.array([])
sift_kps = list()
for data in train_data:
    img = data
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    sift_kps.append(kp)
    
    cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    sift_train_data = np.append(sift_train_data, img)

showImage(sift_train_data[0])

ImportError: cannot import name '_path'